In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
import os
import numpy as np
import random
import librosa
import h5py
import tensorflow as tf
import keras
from keras.optimizers import Adam
from l3embedding.audio import pcm2float
from resampy import resample
import pescador
from skimage import img_as_float
from tqdm import tqdm

Using TensorFlow backend.


In [3]:
def shuffle_files(iterable):
    lst = list(iterable)
    random.shuffle(lst)
    return iter(lst)

In [4]:
def amplitude_to_db(S, amin=1e-10, dynamic_range=80.0):
    magnitude = np.abs(S)
    power = np.square(magnitude, out=magnitude)
    ref_value = power.max()

    log_spec = 10.0 * np.log10(np.maximum(amin, magnitude))
    log_spec -= log_spec.max()

    log_spec = np.maximum(log_spec, -dynamic_range)
    return log_spec

In [5]:
def get_melspectrogram(frame, n_fft=2048, mel_hop_length=242, samp_rate=48000, n_mels=256, fmax=None):
    S = np.abs(librosa.core.stft(frame, n_fft=n_fft, hop_length=mel_hop_length, window='hann', center=True, pad_mode='constant'))
    S = librosa.feature.melspectrogram(sr=samp_rate, S=S, n_fft=n_fft, n_mels=n_mels, fmax=fmax, power=1.0, htk=True)
    S = amplitude_to_db(np.array(S))
    return S

In [6]:
def quant_data_generator(data_dir, batch_size=512, samp_rate=48000, n_fft=2048, \
                         n_mels=256, mel_hop_length=242, hop_size=0.1, fmax=None,\
                         random_state=None, start_batch_idx=None):

    if random_state:
        random.seed(random_state)
        
    frame_length = samp_rate * 1

    batch = None
    curr_batch_size = 0
    batch_idx = 0
       
    for fname in shuffle_files(os.listdir(data_dir)):
        print(fname)
        data_batch_path = os.path.join(data_dir, fname)
        blob_start_idx = 0

        data_blob = h5py.File(data_batch_path, 'r')
        blob_size = len(data_blob['audio'])

        while blob_start_idx < blob_size:
            blob_end_idx = min(blob_start_idx + batch_size - curr_batch_size, blob_size)

            # If we are starting from a particular batch, skip computing all of
            # the prior batches
            if start_batch_idx is None or batch_idx >= start_batch_idx:
                if batch is None:
                    batch = data_blob['audio'][blob_start_idx:blob_end_idx]
                else:
                    batch = np.concatenate([batch, data_blob['audio'][blob_start_idx:blob_end_idx]])

            curr_batch_size += blob_end_idx - blob_start_idx
            blob_start_idx = blob_end_idx

            if blob_end_idx == blob_size:
                data_blob.close()

            if curr_batch_size == batch_size:
                X = []
                # If we are starting from a particular batch, skip yielding all
                # of the prior batches
                if start_batch_idx is None or batch_idx >= start_batch_idx:
                    # Convert audio to float
                    if(samp_rate==48000):
                        batch = pcm2float(batch, dtype='float32')
                    else:
                        batch = resample(pcm2float(batch, dtype='float32'), sr_orig=48000,
                                                  sr_new=samp_rate)

                    X = [get_melspectrogram(batch[i].flatten(), n_fft=n_fft, mel_hop_length=mel_hop_length,\
                                            samp_rate=samp_rate, n_mels=n_mels, fmax=fmax) for i in range(batch_size)]

                    batch = np.array(X)[:, :, :, np.newaxis]
                    #print(np.shape(batch)) #(64, 256, 191, 1)
                    return batch

                batch_idx += 1
                curr_batch_size = 0
                batch = None

In [7]:
def single_epoch_test_data_generator(data_dir, epoch_size, **kwargs):
    for _ in range(epoch_size):
        x = quant_data_generator(data_dir, **kwargs)
        yield x

In [14]:
def keras_for_tflite(model_path, output_path):
    keras.backend.clear_session()
    keras.backend.set_learning_phase(0)
    l3model = keras.models.load_model(model_path)
    embed_layer = l3model.get_layer('audio_embedding_layer')
    pool_size = tuple(embed_layer.get_output_shape_at(0)[1:3])
    y_a = keras.layers.MaxPooling2D(pool_size=pool_size, padding='same')(l3model.output)    
    model = keras.models.Model(inputs=l3model.input, outputs=y_a)
    model.save(output_path)

    return model 

In [9]:
def quantize_keras_to_tflite(tflite_model_file, keras_model_path, quant_mode='default',\
                             calibrate_data_dir=None, num_calibration_steps=1024):

    def representative_dataset_gen():
            l3_model = os.path.dirname(tflite_model_file)
            splits = l3_model.split('_')
            samp_rate = int(splits[3])
            n_mels = int(splits[4])
            mel_hop_length = int(splits[5])
            n_fft = int(splits[-1])

            print('Calibrating.........')
            for _ in range(num_calibration_steps):
                x = quant_data_generator(calibrate_data_dir, batch_size=1,\
                                         samp_rate=samp_rate, n_fft=n_fft,\
                                         n_mels=n_mels, mel_hop_length=mel_hop_length)
                yield [np.array(x).astype(np.float32)]
                
    converter = tf.lite.TFLiteConverter.from_keras_model_file(keras_model_path)
    
    if quant_mode == 'default':
        if calibrate_data_dir is None:
            raise ValueError('Quantized activation calibration needs data directory!')
        
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.default_ranges_stats = (0, 1)
        converter.representative_dataset = representative_dataset_gen
        
    elif quant_mode == 'size':
        converter.post_training_quantize = True
        converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
    else:
        raise ValueError('Unrecognized Quantization mode!')

    tflite_model = converter.convert()
    with open(tflite_model_file, "wb") as f:
        f.write(tflite_model)

In [10]:
def get_embeddings_batch_from_tflite(data_gen, tflite_model_file, epoch_size, batch_size, emb_len=512):
    
    predictions = np.zeros(shape=(epoch_size, batch_size, emb_len))
    #original_embeddings = np.zeros(shape=(epoch_size, batch_size, emb_len))

    interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_shape = input_details[0]['shape'][1:]
    output_shape = output_details[0]['shape'][1:]
    input_index = input_details[0]['index']
    output_index = output_details[0]['index']

    interpreter.resize_tensor_input(input_index, ((batch_size, ) + tuple(input_shape)))
    interpreter.resize_tensor_input(output_index, ((batch_size, ) + tuple(input_shape)))
    interpreter.allocate_tensors()
    
    print("== Input details ==")
    print(interpreter.get_input_details()[0])
    print("type:", input_details[0]['dtype'])
    print("\n== Output details ==")
    print(interpreter.get_output_details()[0])
                
    #predictions per batch   
    for idx, batch_x in enumerate(data_gen):
        x = np.array(batch_x).astype(np.float32)
        interpreter.set_tensor(input_index, x)
        interpreter.invoke()
        output = interpreter.get_tensor(output_index)
        flattened_output = np.reshape(output, (output.shape[0], output.shape[-1]))
        predictions[idx, :, :] = flattened_output
        
    return predictions

In [11]:
def gen_embedding(model_path, tflite_model_file, data_dir, quant_mode='default',\
                  emb_len=512, batch_size=64, epoch_size=1024):
    
    output = None
    print('Getting embedding out of Quantized tflite model')
    splits = os.path.basename(model_path).strip('.h5').split('_')
    samp_rate = int(splits[3])
    n_mels = int(splits[4])
    mel_hop_length = int(splits[5])
    n_fft = int(splits[-1])
    
    data_gen = single_epoch_test_data_generator(data_dir, epoch_size,\
                                                batch_size=batch_size, samp_rate=samp_rate,\
                                                n_fft=n_fft, n_mels=n_mels, mel_hop_length=mel_hop_length)

    output = get_embeddings_batch_from_tflite(data_gen, tflite_model_file,\
                                              epoch_size, batch_size, emb_len=emb_len)
    return output

In [12]:
def post_training_quantization(model_path, calibrate_data_dir, quant_mode='default',\
                               calibration_steps=1024):
    
    #1. Convert l3model to keras model for quantization (with maxpooling layer but flatten removed)
    dir_prefix = '/scratch/sk7898/quantization/' + os.path.basename(model_path).strip('.h5')
    
    if not os.path.isdir(dir_prefix):
        os.makedirs(dir_prefix)
    
    #print('Saving keras model for Quantization')
    keras_model_path = os.path.join(dir_prefix, 'for_quantization.h5')    
    #keras_model = keras_for_tflite(model_path, keras_model_path)
    
    #2.1 Convert keras to tflite model
    #2.2 Quantize model with mode 'default' for only weights quantization or 'size' for full quantization
    #2.3 Save the quantized tflite model
    
    print('Quantizing keras model and saving as tflite')
    tflite_model_file = os.path.join(dir_prefix, 'quantized_model_copy_'+ quant_mode + '.tflite')
    
    quantize_keras_to_tflite(tflite_model_file, keras_model_path, quant_mode=quant_mode,\
                             calibrate_data_dir=calibrate_data_dir, num_calibration_steps=calibration_steps)

In [13]:
model_path = '/scratch/sk7898/l3pruning/embedding/fixed/reduced_input/l3_audio_original_48000_256_242_2048.h5'
calibrate_data_dir = '/beegfs/work/AudioSetSamples/music_train'
calibration_steps = 256
quant_mode = 'default'

post_training_quantization(model_path, calibrate_data_dir, quant_mode=quant_mode, \
                                        calibration_steps=calibration_steps)

Quantizing keras model and saving as tflite
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 48 variables.
INFO:tensorflow:Converted 48 variables to const ops.
Calibrating.........
20187035_3_50.h5


/scratch/sk7898/miniconda3/envs/l3embedding-tf-14-gpu/lib/python3.6/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


20183025_1_74.h5
20187031_3_64.h5
20180306_4_59.h5
20183048_0_32.h5
20187030_2_97.h5
20187034_2_69.h5
20183035_3_13.h5
20180288_2_132.h5
20187019_3_43.h5
20180292_6_165.h5
20187009_1_1.h5
20180263_1_122.h5
20180144_2_65.h5
20183040_0_17.h5
20183051_3_224.h5
20180186_4_189.h5
20183002_2_11.h5
20180202_4_2.h5
20180221_7_160.h5
20180159_1_152.h5
20180203_5_168.h5
20180205_7_143.h5
20180196_6_64.h5
20183020_0_110.h5
20187035_3_20.h5
20180188_6_122.h5
20183088_0_303.h5
20183043_3_91.h5
20180270_0_123.h5
20183074_2_39.h5
20180185_3_48.h5
20183074_2_134.h5
20180203_5_1.h5
20180310_0_89.h5
20183085_1_126.h5
20183054_2_134.h5
20180255_1_111.h5
20183050_2_181.h5
20183001_1_104.h5
20183078_2_68.h5
20180211_5_105.h5
20183031_3_46.h5
20180297_3_71.h5
20183046_2_73.h5
20187020_0_50.h5
20183030_2_80.h5
20183068_0_50.h5
20183065_1_129.h5
20180186_4_193.h5
20180254_0_12.h5
20180181_7_122.h5
20185089_1_62.h5
20180282_4_94.h5
20183081_1_67.h5
20180147_5_67.h5
20183067_3_159.h5
20180257_3_46.h5
20183067_3

20187039_3_41.h5
20180188_6_111.h5
20180151_1_40.h5
20185056_0_45.h5
20180218_4_85.h5
20183032_0_8.h5
20180283_5_1.h5
20187003_3_50.h5
20180285_7_120.h5
20183008_0_115.h5
20180265_3_37.h5
20187021_1_108.h5
20183094_2_177.h5
20183020_0_61.h5
20180185_3_38.h5
20183060_0_158.h5
20185084_0_114.h5
20180285_7_86.h5
20183069_1_140.h5
20180274_4_33.h5
20180173_7_61.h5
20180203_5_74.h5
20183088_0_298.h5
20180220_6_51.h5
20180305_3_179.h5
20183023_3_151.h5
20180292_6_153.h5
20183069_1_71.h5
20180141_7_39.h5
20185086_2_11.h5
20183066_2_206.h5
20185060_0_2.h5
20180298_4_28.h5
20183076_0_36.h5
20185070_2_75.h5
20183061_1_270.h5
20180308_6_159.h5
20183063_3_11.h5
20183083_3_227.h5
20180195_5_179.h5
20183002_2_108.h5
20180188_6_44.h5
20183088_0_147.h5
20183002_2_23.h5
20183089_1_88.h5
20183092_0_194.h5
20185058_2_19.h5
20180255_1_164.h5
20183071_3_299.h5
20180129_3_57.h5
20180314_4_200.h5
20183040_0_134.h5
20180191_1_13.h5
20185079_3_105.h5
20183048_0_328.h5
20180153_3_58.h5
20183091_3_22.h5
20185052

20180265_3_50.h5
20187030_2_36.h5
20180217_3_187.h5
20183007_3_68.h5
20183088_0_314.h5
20187041_1_63.h5
20183087_3_326.h5
20180164_6_162.h5
20180283_5_76.h5
20180205_7_101.h5
20180133_7_36.h5
20183061_1_149.h5
20180160_2_32.h5
20183036_0_29.h5
20187022_2_1.h5
20183043_3_78.h5
20183021_1_59.h5
20180299_5_93.h5
20187046_2_33.h5
20187003_3_62.h5
20185061_1_77.h5
20180179_5_88.h5
20180221_7_18.h5
20180133_7_34.h5
20180169_3_104.h5
20180177_3_218.h5
20183091_3_111.h5
20183010_2_122.h5
20185060_0_29.h5
20180281_3_24.h5
20180199_1_43.h5
20180136_2_18.h5
20187006_2_95.h5
20187043_3_73.h5
20180145_3_13.h5
20183079_3_220.h5
20183067_3_281.h5
20185090_2_80.h5
20180209_3_63.h5
20180207_1_103.h5
20187025_1_114.h5
20180307_5_122.h5
20180280_2_87.h5
20180271_1_114.h5
20180316_6_159.h5
20185083_3_117.h5
20183084_0_220.h5
20183070_2_181.h5
20183040_0_47.h5
20180262_0_104.h5
20180280_2_19.h5
20187031_3_93.h5
20183008_0_33.h5
20183025_1_20.h5
20180269_7_82.h5
20185065_1_13.h5
20180306_4_38.h5
20183061_1_

In [ ]:
model_path = '/scratch/sk7898/l3pruning/embedding/fixed/reduced_input/l3_audio_original_48000_256_242_2048.h5'
tflite_model_file = '/scratch/sk7898/quantization/l3_audio_original_48000_256_242_2048/quantized_model_default.tflite'
data_dir = '/beegfs/work/AudioSetSamples/music_train'

embeddings = gen_embedding(model_path, tflite_model_file, data_dir, quant_mode='default',\
                           batch_size=64, epoch_size=64)

Getting embedding out of Quantized tflite model
== Input details ==
{'name': 'input_13', 'index': 33, 'shape': array([ 64, 256, 199,   1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}
type: <class 'numpy.float32'>

== Output details ==
{'name': 'max_pooling2d_1/MaxPool', 'index': 34, 'shape': array([ 64,   1,   1, 512], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}
20183047_3_31.h5


/scratch/sk7898/miniconda3/envs/l3embedding-tf-14-cpu/lib/python3.6/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
